# VCG mechanism in Bayesian ad auctions with signaling

## Preliminars

The model setting for ad auctions is:

* Players: $N = \{1, ..., n\}$
* Ads: $A = \{a_1, ..., a_n\}$
* Type space: $Q = \{q_1,..., q_n:q_i \in [0,1]\}$ (Value that a player gives to a click in their ad)
* Slots: $S = \{s_1,...,s_k\}$
* Slot prominence: $Λ = \{λ_1,...,λ_k:  λ_i \in [0,1], λ_i \geq λ_{i+1}\}$
* Valuation function: $v_i(k,\theta_i) = \begin{cases}
    λ_j*q_i*θ_i,& \text{if } a_i \text{ allocated in } s_j \text{ is in } k\\
    0,              & \text{otherwise}
\end{cases}$

## Our setting
 > Notation may change

* Players (Bidders): $N = \{1, ..., n\}$
* Slots: $M = \{m_1, ..., m_m\}$; where m < n
* Slot prominence: $Λ = \{λ_1, ..., λ_k:  λ_i \in [0,1], λ_i \geq λ_{i+1}\}$ (In this setting $λ_i$ corresponds to the click through rate which in the previous setting was $λ_i*q_i$)
* States of nature: $Σ = \{σ_1, ..., σ_d\}$
* Type space: $Θ = \{θ_1(σ), ..., θ_n(σ):θ_i \in [0,1]\}$  (Value that a player gives to a click in their ad depending on the state of nature)
* Valuation function: $v_i(k,\theta_i) = \begin{cases}
    λ_j*θ_i,& \text{if } a_i \text{ allocated in } s_j \text{ is in } k\\
    0,              & \text{otherwise}
\end{cases}$
* Signals: $S = \{s_1, ..., s_h\}$
* Signaling scheme $ϕ: Σ \to Δ_S$
* Common prior distribution of states: $\mu$ where $\mu \in \Delta_Σ$

We assume that: 
- $\sigma \sim \mu$. 
- $\mu_\sigma = prob_\mu(\sigma)\ \ ∀ σ \in Σ$. 
- $ \phi_\sigma(s)$ is the probability of sending signal $s$ when the state of nature is $\sigma \in Σ$.
- The signal is public, therefore all bidders have access to it.

The interaction is as follows:

![timeline.png](<./timeline.png>)

**Computation of posterior distribution**

The bidders compute their posterior distribution $\xi$ given the signal using bayes rule:
\begin{equation}
\xi_s(\sigma) = \frac{\mu_\sigma \phi_\sigma(s)}{\sum_{\sigma' \in \Sigma} \mu_{\sigma'} \phi_{\sigma'}(s)}.
\end{equation}

Since we are considering the case in which the auctioneer uses a VCG mechanism the bidders will report it's expected value given $\xi$, so each bidders reports:

\begin{equation}
\xi_s^\top \theta_i = \sum_{\sigma \in \Sigma} \theta_i(\sigma) \xi_s(\sigma).
\end{equation}

The total overall revenue of the auctionee is:

\begin{equation}
\text{Rev}(\Theta, \xi) = \sum_{j=1}^m j \left( \sum_{\sigma \in \Sigma} \xi_s(\sigma) \theta_{j+1}(\sigma) \right) (\lambda_j - \lambda_{j+1}),
\end{equation}

where $\lambda_{m+1} = 0$

> The signaling scheme is a degree of freedom that the auctioneer can use to maximize revenue





In [1]:
from amplpy import AMPL
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import math
from itertools import permutations

In [2]:
def posterior(signal, state, Prior, Likelihood, States):
    return Prior.loc[state, 0]*Likelihood.loc[state, signal]/sum([Prior.loc[state_p, 0]*Likelihood.loc[state_p, signal] for state_p in States])  

def bid(bidder, Posterior, Theta, States):
    return sum([Theta.loc[bidder, state]*Posterior[state] for state in States])

def allocate(Bids, Slots):
    ordered_bids = sorted(Bids, reverse=True)
    return [ordered_bids[bid] for bid in range(len(Slots))]

def order_theta(Theta_df, Bids):
    Theta_df['Bid'] = Bids
    Theta_df = Theta_df.sort_values(by='Bid', ascending=False)
    Theta_df = Theta_df.drop(columns=['Bid'])
    Theta_df = Theta_df.reset_index(drop=True)
    return Theta_df

def revenue(Slots, States, Posterior, Theta, Promminence):
    Promminence.append(0)
    revenue = 0
    for j in range(len(Slots)):
        inner_sum = 0
        for state in States:
            inner_sum += Posterior[state]*Theta.loc[j+1, state]
        revenue += (j+1)*inner_sum*(Promminence[j] - Promminence[j+1])
    return revenue

In [3]:
def gamma(Posterior, Signals, States, Prior, Likelihood):
    gamma = 0
    for signal in Signals:
        inner_sum = 0
        for state in States:
            inner_sum += Prior.loc[state, 0]*Likelihood.loc[state, signal]
        gamma += inner_sum if [posterior(signal, state_p, Prior, Likelihood, States) for state_p in States] == Posterior else 0
        
    return gamma

def check_gamma(States, Posteriors, Signals, Prior, Likelihood):
    for state in States:
        mu = 0
        for posterior in Posteriors:
            mu += posterior[state]*gamma(posterior, Signals, States, Prior, Likelihood) 
        if mu != Prior.loc[state, 0]:
            raise ValueError('Gamma not good')

In [4]:
Bidders = [0, 1]
States = [0, 1, 2]
Slots = [0]
Promminence = [1] 
Prior = [1/len(States) for i in States] # uniform prior
Theta = [[0.9, 0.1, 0.9], 
        [0.9, 0.9, 0.1]]

Prior_df = pd.DataFrame(Prior, index=States)
Theta_df = pd.DataFrame(Theta, index=Bidders, columns=States)

> ϕ can be represented as a matrix
 
## **No strategic signaling**: 
### **State is revealed**
  - The is the same amount of signals than states
  - ϕ is a square diagonal matrix

In [5]:
# Truthful state
Signals = States.copy()

Phi = np.diag([1 for i in States])
Phi_df = pd.DataFrame(Phi, index=States, columns=Signals)

### **Timeline**

* $\theta \sim \mu$
* $s \sim \phi_\theta$
* Compute posterior
* Give bid
* Allocate ad
* Calculate Revenue

In [6]:
signal = 0

Posterior = [posterior(signal, state, Prior_df, Phi_df, States) for state in States]
Bids = [bid(bidder, Posterior, Theta_df, States) for bidder in Bidders]
Allocation = allocate(Bids, Slots)
Theta_df_ordered = order_theta(Theta_df, Bids)
revenue_0 = revenue(Slots, States, Posterior, Theta_df_ordered, Promminence)

print("Revenue when the state is 0:",revenue_0)
print("Posterior:", Posterior)
print("Bids:", Bids)
print("Allocation:", Allocation)

Revenue when the state is 0: 0.9
Posterior: [1.0, 0.0, 0.0]
Bids: [0.9, 0.9]
Allocation: [0.9]


In [7]:
signal = 1

Posterior = [posterior(signal, state, Prior_df, Phi_df, States) for state in States]
Bids = [bid(bidder, Posterior, Theta_df, States) for bidder in Bidders]
Allocation = allocate(Bids, Slots)
Theta_df_ordered = order_theta(Theta_df, Bids)
revenue_1 = revenue(Slots, States, Posterior, Theta_df_ordered, Promminence)

print("Revenue when the state is 1:",revenue_1)
print("Posterior:", Posterior)
print("Bids:", Bids)
print("Allocation:", Allocation)

Revenue when the state is 1: 0.1
Posterior: [0.0, 1.0, 0.0]
Bids: [0.1, 0.9]
Allocation: [0.9]


In [8]:
signal = 2

Posterior = [posterior(signal, state, Prior_df, Phi_df, States) for state in States]
Bids = [bid(bidder, Posterior, Theta_df, States) for bidder in Bidders]
Allocation = allocate(Bids, Slots)
Theta_df_ordered = order_theta(Theta_df, Bids)
revenue_2 = revenue(Slots, States, Posterior, Theta_df_ordered, Promminence)

print("Revenue when the state is 2:", revenue_2)
print("Posterior:", Posterior)
print("Bids:", Bids)
print("Allocation:", Allocation)

Revenue when the state is 2: 0.1
Posterior: [0.0, 0.0, 1.0]
Bids: [0.9, 0.1]
Allocation: [0.9]


In [9]:
expected_revenue = sum([revenue_i*Prior[i] for i, revenue_i in enumerate([revenue_0, revenue_1, revenue_2])])
print("Expected revenue:", expected_revenue)

Expected revenue: 0.36666666666666664


### All together

In [10]:
def expected_revenue(States, Bidders, Slots, Theta_df, Promminence, Signals, Prior_df, Phi_df):
    Posteriors = [[posterior(signal, state, Prior_df, Phi_df, States) for state in States] for signal in Signals]
    expected_revenue = 0

    # Test
    check_gamma(States, Posteriors, Signals, Prior_df, Phi_df)
    
    for Posterior in Posteriors:
        Bids = [bid(bidder, Posterior, Theta_df, States) for bidder in Bidders]
        # print("Bids",Bids)
        Theta_df_ordered = order_theta(Theta_df, Bids)
        rev = revenue(Slots, States, Posterior, Theta_df_ordered, Promminence)
        # print("Revenue",rev)
        expected_revenue += gamma(Posterior, Signals, States, Prior_df, Phi_df)*rev
        # print("Gamma",gamma(Posterior, Signals, States, Prior_df, Phi_df))
        print("")
    return expected_revenue

In [11]:
expected_rev = expected_revenue(States, Bidders, Slots, Theta_df, Promminence, Signals, Prior_df, Phi_df)
print("Expected revenue:", expected_rev)




Expected revenue: 0.36666666666666664


### **State is not revealed**

In [12]:
# No information about the state
Signals = [0]

Phi = [1 for j in States]
Phi_df = pd.DataFrame(Phi, index=States, columns=Signals)

In [13]:
expected_rev = expected_revenue(States, Bidders, Slots, Theta_df, Promminence, Signals, Prior_df, Phi_df)
print("Expected revenue:", expected_rev)


Expected revenue: 0.6333333333333333


## **Strategic signaling**: 

In [14]:
def compute_optimal_signaling_scheme(States, index_permutations, Slots, v_df, Promminence, Prior):    
    Promminence.append(0)
    m = len(Slots)
    
    ampl = AMPL()
    ampl.read('model_KV.mod')
    
    ampl.set["STATES"] = States
    ampl.set["PI"] = index_permutations
    
    ampl.param["m"] = m
    ampl.param["v"] = v_df
    ampl.param["lambda"] = Promminence
    ampl.param["mu"] = Prior
    
    ampl.solve(solver='gurobi')
    assert ampl.solve_result == "solved"
    
    obj = ampl.getObjective('obj').value()
    x = ampl.getVariable('x').getValues().toPandas()
    ampl.close()
    
    return obj, x

In [15]:
def P(Bidders, Slots):
    res = list(permutations(Bidders, len(Slots)))
    res = [list(t) for t in res]
    return res

def build_v_df(Permutations, Theta_df):
    v = []
    for permutation in Permutations:
        temp = Theta_df.loc[permutation, :]
        temp = temp.reset_index(drop=True)
        v.append(temp)
    index = [i for i in range(len(Permutations))]
    return pd.concat(v, keys = index)

In [16]:
# Slots = [0, 1] # Slots
# Bidders = [0, 1, 2] # Bidders
Bidders = [0, 1]
States = [0, 1, 2]
Slots = [0]
Promminence = [1] 
Prior = [1/len(States) for i in States] # uniform prior
Theta = [[0.9, 0.1, 0.9], 
        [0.9, 0.9, 0.1]]

Prior_df = pd.DataFrame(Prior, index=States)
Theta_df = pd.DataFrame(Theta, index=Bidders, columns=States)

# Maximum revenue with optimal signaling scheme
n_players = len(Bidders)
m_slots = len(Slots)

Slots_plus = Slots.copy()
Slots_plus.append(len(Slots))
Permutations = P(Bidders, Slots_plus)
index_permutations = [i for i in range(len(Permutations))]
v_df = build_v_df(Permutations, Theta_df)

obj, x = compute_optimal_signaling_scheme(States, index_permutations, Slots, v_df, Promminence, Prior)

print("Result:")
print("Objective value:", obj)
print("X:", x)

Gurobi 11.0.1:Gurobi 11.0.1: optimal solution; objective 0.6333333333
0 simplex iterations
Result:
Objective value: 0.6333333333333333
X:                   x.val
index0 index1          
0      0       0.333333
       1       0.333333
       2       0.333333
1      0       0.000000
       1       0.000000
       2       0.000000


In [17]:
pers = x.index.get_level_values(0).unique().values
states = x.index.get_level_values(1).unique().values

# Gamma
gamma  = []
for pi in pers:
    # print("Permutation", Permutations[pi])
    sum = 0
    for state in states:
        sum += x.loc[pi, state].iloc[0]
    gamma.append(sum)
    
print("Gamma:", gamma)

# Posteriors
posteriors = []
for pi in pers:
    if gamma[pi] == 0:
        continue
    
    posterior = []
    for state in states:
        posterior.append(x.loc[pi, state].iloc[0]/gamma[pi])
    posteriors.append(posterior)

print("Posteriors:", posteriors)

Gamma: [1.0, 0.0]
Posteriors: [[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]]


Giving no information about the state to the bidder is the best signaling scheme the auctioneer can do.